In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy.stats import chi2_contingency
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols

In [2]:
df=pd.read_csv('watson_healthcare_modified.csv')

In [3]:
df.shape

(1676, 35)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1676 entries, 0 to 1675
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   EmployeeID                1676 non-null   int64 
 1   Age                       1676 non-null   int64 
 2   Attrition                 1676 non-null   object
 3   BusinessTravel            1676 non-null   object
 4   DailyRate                 1676 non-null   int64 
 5   Department                1676 non-null   object
 6   DistanceFromHome          1676 non-null   int64 
 7   Education                 1676 non-null   int64 
 8   EducationField            1676 non-null   object
 9   EmployeeCount             1676 non-null   int64 
 10  EnvironmentSatisfaction   1676 non-null   int64 
 11  Gender                    1676 non-null   object
 12  HourlyRate                1676 non-null   int64 
 13  JobInvolvement            1676 non-null   int64 
 14  JobLevel                

In [5]:
df.describe()

,EmployeeID,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,...,RelationshipSatisfaction,StandardHours,Shift,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
count,1.676000e+03,1676.000000,1676.000000,1676.000000,1676.000000,1676.0,1676.000000,1676.000000,1676.000000,1676.000000,...,1676.000000,1676.0,1676.000000,1676.000000,1676.000000,1676.000000,1676.000000,1676.000000,1676.000000,1676.000000
mean,1.456796e+06,36.866348,800.557876,9.221957,2.907518,1.0,2.714797,65.470167,2.724940,2.066826,...,2.718377,80.0,0.806086,11.338902,2.805489,2.766110,7.033413,4.264916,2.200477,4.135442
std,2.487486e+05,9.129126,401.594438,8.158118,1.025835,0.0,1.097534,20.207572,0.714121,1.113423,...,1.078162,0.0,0.855527,7.834996,1.288431,0.702369,6.098991,3.627456,3.229587,3.559662
min,1.025177e+06,18.000000,102.000000,1.000000,1.000000,1.0,1.000000,30.000000,1.000000,1.000000,...,1.000000,80.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,1.235832e+06,30.000000,465.000000,2.000000,2.000000,1.0,2.000000,48.000000,2.000000,1.000000,...,2.000000,80.0,0.000000,6.000000,2.000000,2.000000,3.000000,2.000000,0.000000,2.000000
50%,1.464606e+06,36.000000,796.500000,7.000000,3.000000,1.0,3.000000,65.500000,3.000000,2.000000,...,3.000000,80.0,1.000000,10.000000,3.000000,3.000000,5.000000,3.000000,1.000000,3.000000
75%,1.667992e+06,43.000000,1157.000000,14.000000,4.000000,1.0,4.000000,83.000000,3.000000,3.000000,...,4.000000,80.0,1.000000,15.000000,3.000000,3.000000,10.000000,7.000000,3.000000,7.000000
max,1.886378e+06,60.000000,1499.000000,29.000000,5.000000,1.0,4.000000,100.000000,4.000000,5.000000,...,4.000000,80.0,3.000000,40.000000,6.000000,4.000000,40.000000,18.000000,15.000000,17.000000


In [6]:
pd.set_option('display.max_columns',None)
df.head()

,EmployeeID,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,Shift,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,1313919,41,No,Travel_Rarely,1102,Cardiology,1,2,Life Sciences,1,2,Female,94,3,2,Nurse,4,Single,5993,19479,8,Y,Yes,11,3,1,80,0,8,0,1,6,4,0,5
1,1200302,49,No,Travel_Frequently,279,Maternity,8,1,Life Sciences,1,3,Male,61,2,2,Other,2,Married,5130,24907,1,Y,No,23,4,4,80,1,10,3,3,10,7,1,7
2,1060315,37,Yes,Travel_Rarely,1373,Maternity,2,2,Other,1,4,Male,92,2,1,Nurse,3,Single,2090,2396,6,Y,Yes,15,3,2,80,0,7,3,3,0,0,0,0
3,1272912,33,No,Travel_Frequently,1392,Maternity,3,4,Life Sciences,1,4,Female,56,3,1,Other,3,Married,2909,23159,1,Y,Yes,11,3,3,80,0,8,3,3,8,7,3,0
4,1414939,27,No,Travel_Rarely,591,Maternity,2,1,Medical,1,1,Male,40,3,1,Nurse,2,Married,3468,16632,9,Y,No,12,3,4,80,1,6,3,3,2,2,2,2


In [7]:
df.drop('EmployeeID',inplace=True,axis=1)

In [8]:
df.isnull().sum()

Age                         0
Attrition                   0
BusinessTravel              0
DailyRate                   0
Department                  0
DistanceFromHome            0
Education                   0
EducationField              0
EmployeeCount               0
EnvironmentSatisfaction     0
Gender                      0
HourlyRate                  0
JobInvolvement              0
JobLevel                    0
JobRole                     0
JobSatisfaction             0
MaritalStatus               0
MonthlyIncome               0
MonthlyRate                 0
NumCompaniesWorked          0
Over18                      0
OverTime                    0
PercentSalaryHike           0
PerformanceRating           0
RelationshipSatisfaction    0
StandardHours               0
Shift                       0
TotalWorkingYears           0
TrainingTimesLastYear       0
WorkLifeBalance             0
YearsAtCompany              0
YearsInCurrentRole          0
YearsSinceLastPromotion     0
YearsWithC

## Smote

In [9]:
df.Attrition.value_counts()

No     1477
Yes     199
Name: Attrition, dtype: int64

In [10]:
no=df[df.Attrition=='No']
yes=df[df.Attrition=='Yes']

from sklearn.utils import resample

over=resample(yes,replace=True,n_samples=len(no),random_state=23)

dfsmote=pd.concat([no,over])
dfsmote.Attrition.value_counts()

No     1477
Yes    1477
Name: Attrition, dtype: int64

In [11]:
df.shape

(1676, 34)

# 5. Data Preprocessing

In [12]:

x = dfsmote.drop("Attrition",axis = 1)
y =  dfsmote["Attrition"]

In [13]:
x.shape

(2954, 33)

In [14]:
x.head()

,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,Shift,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Travel_Rarely,1102,Cardiology,1,2,Life Sciences,1,2,Female,94,3,2,Nurse,4,Single,5993,19479,8,Y,Yes,11,3,1,80,0,8,0,1,6,4,0,5
1,49,Travel_Frequently,279,Maternity,8,1,Life Sciences,1,3,Male,61,2,2,Other,2,Married,5130,24907,1,Y,No,23,4,4,80,1,10,3,3,10,7,1,7
3,33,Travel_Frequently,1392,Maternity,3,4,Life Sciences,1,4,Female,56,3,1,Other,3,Married,2909,23159,1,Y,Yes,11,3,3,80,0,8,3,3,8,7,3,0
4,27,Travel_Rarely,591,Maternity,2,1,Medical,1,1,Male,40,3,1,Nurse,2,Married,3468,16632,9,Y,No,12,3,4,80,1,6,3,3,2,2,2,2
5,32,Travel_Frequently,1005,Maternity,2,2,Life Sciences,1,4,Male,79,3,1,Nurse,4,Single,3068,11864,0,Y,No,13,3,3,80,0,8,2,2,7,7,3,6


In [15]:

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x[['MonthlyRate','MonthlyIncome','HourlyRate','Age','DailyRate']] = scaler.fit_transform(x[['MonthlyRate','Age','MonthlyIncome','HourlyRate','DailyRate']])

In [16]:
x

,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,Shift,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,0.914286,Travel_Rarely,0.715820,Cardiology,1,2,Life Sciences,1,2,Female,0.262454,3,2,Nurse,4,Single,0.547619,0.698053,8,Y,Yes,11,3,1,80,0,8,0,1,6,4,0,5
1,0.442857,Travel_Frequently,0.126700,Maternity,8,1,Life Sciences,1,3,Male,0.217009,2,2,Other,2,Married,0.738095,0.916001,1,Y,No,23,4,4,80,1,10,3,3,10,7,1,7
3,0.371429,Travel_Frequently,0.923407,Maternity,3,4,Life Sciences,1,4,Female,0.100053,3,1,Other,3,Married,0.357143,0.845814,1,Y,Yes,11,3,3,80,0,8,3,3,8,7,3,0
4,0.142857,Travel_Rarely,0.350036,Maternity,2,1,Medical,1,1,Male,0.129489,3,1,Nurse,2,Married,0.214286,0.583738,9,Y,No,12,3,4,80,1,6,3,3,2,2,2,2
5,0.700000,Travel_Frequently,0.646385,Maternity,2,2,Life Sciences,1,4,Male,0.108425,3,1,Nurse,4,Single,0.333333,0.392291,0,Y,No,13,3,3,80,0,8,2,2,7,7,3,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1278,0.771429,Travel_Rarely,0.143880,Cardiology,27,3,Life Sciences,1,3,Male,0.252975,3,2,Nurse,4,Single,0.404762,0.457659,1,Y,Yes,18,3,4,80,0,10,2,3,10,7,7,7
456,0.557143,Travel_Frequently,0.861847,Cardiology,5,3,Marketing,1,2,Male,0.045761,3,1,Other,2,Single,0.000000,0.239510,1,Y,Yes,14,3,4,80,0,0,3,3,0,0,0,0
1080,0.800000,Travel_Rarely,0.410880,Maternity,9,4,Life Sciences,1,4,Male,0.053133,3,1,Nurse,1,Single,0.523810,0.792491,3,Y,No,14,3,2,80,0,15,3,1,5,4,1,0
51,0.285714,Travel_Rarely,0.953472,Maternity,5,4,Technical Degree,1,3,Male,0.128067,3,1,Nurse,3,Single,0.238095,0.364786,1,Y,Yes,13,3,3,80,0,2,3,2,2,2,2,2


In [17]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [18]:
df.Attrition=le.fit_transform(df.Attrition)

In [19]:
p=pd.get_dummies(x.BusinessTravel,drop_first=True)
o=pd.get_dummies(x.Department,drop_first=True)
i=pd.get_dummies(x.JobRole,drop_first=True)
u=pd.get_dummies(x.OverTime,drop_first=True)
y=pd.get_dummies(x.MaritalStatus,drop_first=True)
f=pd.get_dummies(x.JobRole,drop_first=True)
h=pd.get_dummies(x.OverTime,drop_first=True)
j=pd.get_dummies(x.MaritalStatus,drop_first=True)

In [20]:
x=pd.concat([p,o,i,u,f,j,h,df],axis=1)

In [21]:
x.drop(['BusinessTravel','Department','JobRole','OverTime','MaritalStatus','Over18','Gender','EducationField'],axis=1,inplace=True)

In [22]:
x.head()

,Travel_Frequently,Travel_Rarely,Maternity,Neurology,Administrative,Nurse,Other,Therapist,Yes,Administrative,Nurse,Other,Therapist,Married,Single,Yes,Age,Attrition,DailyRate,DistanceFromHome,Education,EmployeeCount,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,MonthlyRate,NumCompaniesWorked,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,Shift,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,1,41,0,1102,1,2,1,2,94,3,2,4,5993,19479,8,11,3,1,80,0,8,0,1,6,4,0,5
1,1,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,49,0,279,8,1,1,3,61,2,2,2,5130,24907,1,23,4,4,80,1,10,3,3,10,7,1,7
3,1,0,1,0,0,0,1,0,1,0,0,1,0,1,0,1,33,0,1392,3,4,1,4,56,3,1,3,2909,23159,1,11,3,3,80,0,8,3,3,8,7,3,0
4,0,1,1,0,0,1,0,0,0,0,1,0,0,1,0,0,27,0,591,2,1,1,1,40,3,1,2,3468,16632,9,12,3,4,80,1,6,3,3,2,2,2,2
5,1,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,32,0,1005,2,2,1,4,79,3,1,4,3068,11864,0,13,3,3,80,0,8,2,2,7,7,3,6


In [23]:
x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2954 entries, 0 to 1165
Data columns (total 42 columns):
 #   Column                    Non-Null Count  Dtype
---  ------                    --------------  -----
 0   Travel_Frequently         2954 non-null   uint8
 1   Travel_Rarely             2954 non-null   uint8
 2   Maternity                 2954 non-null   uint8
 3   Neurology                 2954 non-null   uint8
 4   Administrative            2954 non-null   uint8
 5   Nurse                     2954 non-null   uint8
 6   Other                     2954 non-null   uint8
 7   Therapist                 2954 non-null   uint8
 8   Yes                       2954 non-null   uint8
 9   Administrative            2954 non-null   uint8
 10  Nurse                     2954 non-null   uint8
 11  Other                     2954 non-null   uint8
 12  Therapist                 2954 non-null   uint8
 13  Married                   2954 non-null   uint8
 14  Single                    2954 non-null 

In [54]:
from sklearn.decomposition import PCA
pca=PCA(0.70)
x_pca=pca.fit_transform(x)

In [55]:
x_pca.shape

(2954, 1)

In [56]:
ps = pd.DataFrame(x_pca)
ps.head()

,0
0,-4809.669684
1,-10185.022720
2,-8311.225004
3,-1829.106779
4,2955.596853


In [57]:
from sklearn.model_selection import train_test_split,GridSearchCV

from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn import metrics

In [58]:
xtrain,xtest,ytrain,ytest=train_test_split(x_pca,y,test_size=0.3)

In [59]:
rf=RandomForestClassifier(n_estimators=1000)
rf.fit(xtrain,ytrain)

RandomForestClassifier(n_estimators=1000)

In [60]:
y_pred=rf.predict(xtest)

In [61]:
confusion_matrix(ytest,y_pred)

ValueError: multilabel-indicator is not supported

In [ ]:
rf.score(xtest,ytest)

In [ ]:
print(classification_report(ytest,y_pred))

In [ ]:
prob=rf.predict_proba(xtest)
prob

In [45]:
#roc
fp,tp,th=roc_curve(ytest,y_pred[:,1])
roc_auc = auc(fp, tp) 
print(roc_auc)

ValueError: multilabel-indicator format is not supported

In [46]:
plt.figure()
plt.plot(fp,tp,label='ROC')
plt.plot([0,1],[0,1],'r--')
plt.xlim([0.0,1.05])
plt.ylim([0.0,1.05])
plt.xlabel('fp_rate')
plt.ylabel('tp_rate')
plt.legend(loc='lower right')
plt.show()

NameError: name 'fp' is not defined

<Figure size 640x480 with 0 Axes>

In [47]:
from sklearn.tree import DecisionTreeClassifier


In [48]:
dt=DecisionTreeClassifier(max_depth=1000)
dt.fit(xtrain,ytrain)

DecisionTreeClassifier(max_depth=1000)

In [49]:
y_pred=dt.predict(xtest)

In [50]:
confusion_matrix(ytest,y_pred)

ValueError: multilabel-indicator is not supported

In [51]:
accuracy_score(ytest,y_pred)

0.7237880496054115

In [52]:
#classification Report
print(classification_report(ytest,y_pred))

              precision    recall  f1-score   support

           0       0.70      0.71      0.71       340
           1       0.81      0.81      0.81       386

   micro avg       0.75      0.76      0.76       726
   macro avg       0.75      0.76      0.76       726
weighted avg       0.76      0.76      0.76       726
 samples avg       0.62      0.62      0.62       726



C:\Users\abhay\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\abhay\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [53]:
from sklearn.linear_model import LogisticRegression
lm=LogisticRegression()
lm.fit(xtrain,ytrain)

ValueError: y should be a 1d array, got an array of shape (2067, 2) instead.

In [ ]:
p_x=lm.predict(xtrain)

In [ ]:
lm.score(xtest,ytest)

In [ ]:
from sklearn.metrics import mean_squared_error
np.sqrt(mean_squared_error(ytrain,p_x))